In [8]:
import os
sys.path.append(r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 10\timeseires')

In [9]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [10]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [11]:
def create_lstm():
    input_data = Input(shape=(time_steps, num_features))
    lstm_layer1 = LSTM(8, return_sequences=True)(input_data)
    lstm_layer2 = LSTM(20)(lstm_layer1)
    x = Flatten()(lstm_layer2)
    output_data = Dense(1)(x)
    model = Model(input_data, output_data)
    return model

In [12]:
model1 = create_lstm()
model1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 24, 21)]          0         
                                                                 
 lstm_2 (LSTM)               (None, 24, 8)             960       
                                                                 
 lstm_3 (LSTM)               (None, 20)                2320      
                                                                 
 flatten_1 (Flatten)         (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 21        
                                                                 
Total params: 3,301
Trainable params: 3,301
Non-trainable params: 0
_________________________________________________________________


In [13]:
tensorflow.keras.utils.plot_model(model1 )

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [14]:
checkpoints = r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 10\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 10'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

In [15]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]

In [16]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =create_lstm()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [17]:
import os
path_dataset ='C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 10\\AEP_hourly_2'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

((84907, 21), (24259, 21), (12130, 21))

In [18]:
time_steps=24
num_features=21

In [19]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 3.4734444618225098 sec


In [20]:
epochs = 60
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/60
2653/2653 [==============================] - ETA: 0s - loss: 0.0387 - mae: 0.0387 - mape: 33.1873
Epoch 1: val_loss improved from inf to 0.01981, saving model to C:\Users\musta\Desktop\ML lab\lab 10\E1-cp-0001-loss0.02.h5
2653/2653 [==============================] - 175s 62ms/step - loss: 0.0387 - mae: 0.0387 - mape: 33.1873 - val_loss: 0.0198 - val_mae: 0.0198 - val_mape: 8.7333
Epoch 2/60
2653/2653 [==============================] - ETA: 0s - loss: 0.0162 - mae: 0.0162 - mape: 229.1206
Epoch 2: val_loss improved from 0.01981 to 0.01357, saving model to C:\Users\musta\Desktop\ML lab\lab 10\E1-cp-0002-loss0.01.h5
2653/2653 [==============================] - 166s 63ms/step - loss: 0.0162 - mae: 0.0162 - mape: 229.1206 - val_loss: 0.0136 - val_mae: 0.0136 - val_mape: 6.9618
Epoch 3/60
2336/2653 [=========================>....] - ETA: 20s - loss: 0.0113 - mae: 0.0113 - mape: 244.6428
Epoch 3: val_loss improved from 0.01357 to 0.00935, saving model to C:\Users\musta\Desktop\ML l

Epoch 25/60
2652/2653 [============================>.] - ETA: 0s - loss: 0.0070 - mae: 0.0070 - mape: 32.3764
Epoch 25: val_loss did not improve from 0.00670
2653/2653 [==============================] - 73s 28ms/step - loss: 0.0070 - mae: 0.0070 - mape: 32.3701 - val_loss: 0.0067 - val_mae: 0.0067 - val_mape: 3.0460
Epoch 26/60
2651/2653 [============================>.] - ETA: 0s - loss: 0.0069 - mae: 0.0069 - mape: 3.4392
Epoch 26: val_loss did not improve from 0.00670
2653/2653 [==============================] - 75s 28ms/step - loss: 0.0069 - mae: 0.0069 - mape: 3.4383 - val_loss: 0.0069 - val_mae: 0.0069 - val_mape: 3.1704
Epoch 27/60
2651/2653 [============================>.] - ETA: 0s - loss: 0.0069 - mae: 0.0069 - mape: 21.0036
Epoch 27: val_loss improved from 0.00670 to 0.00642, saving model to C:\Users\musta\Desktop\ML lab\lab 10\E1-cp-0027-loss0.01.h5
2653/2653 [==============================] - 79s 30ms/step - loss: 0.0069 - mae: 0.0069 - mape: 20.9926 - val_loss: 0.0064 - va

2653/2653 [==============================] - 114s 43ms/step - loss: 0.0063 - mae: 0.0063 - mape: 7.4692 - val_loss: 0.0064 - val_mae: 0.0064 - val_mape: 2.7961
Epoch 50/60
2653/2653 [==============================] - ETA: 0s - loss: 0.0064 - mae: 0.0064 - mape: 61.3683
Epoch 50: val_loss did not improve from 0.00605
2653/2653 [==============================] - 97s 37ms/step - loss: 0.0064 - mae: 0.0064 - mape: 61.3683 - val_loss: 0.0065 - val_mae: 0.0065 - val_mape: 2.7712
Epoch 51/60
2652/2653 [============================>.] - ETA: 0s - loss: 0.0063 - mae: 0.0063 - mape: 76.3125
Epoch 51: val_loss did not improve from 0.00605
2653/2653 [==============================] - 110s 41ms/step - loss: 0.0063 - mae: 0.0063 - mape: 76.2968 - val_loss: 0.0064 - val_mae: 0.0064 - val_mape: 2.6802
Epoch 52/60
2653/2653 [==============================] - ETA: 0s - loss: 0.0063 - mae: 0.0063 - mape: 32.7105
Epoch 52: val_loss did not improve from 0.00605
2653/2653 [==============================] - 

In [21]:

model = load_model(r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 10\E1-cp-0043-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 11s 12ms/step
Mean Absolute Error (MAE): 98.17
Median Absolute Error (MedAE): 75.55
Mean Squared Error (MSE): 17560.73
Root Mean Squared Error (RMSE): 132.52
Mean Absolute Percentage Error (MAPE): 0.67 %
Median Absolute Percentage Error (MDAPE): 0.52 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [22]:
checkpoints = r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 10\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'C:\Users\musta\Desktop\ML lab\lab 10\E1-cp-0043-loss0.01.h5'
start_epoch= 44

In [23]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] loading C:\Users\musta\Desktop\ML lab\lab 10\E1-cp-0043-loss0.01.h5...
[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 9.999999747378752e-05


In [24]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/10
2653/2653 [==============================] - ETA: 0s - loss: 0.0057 - mae: 0.0057 - mape: 6.9084
Epoch 1: val_loss improved from inf to 0.00600, saving model to C:\Users\musta\Desktop\ML lab\lab 10\E2-cp-0001-loss0.01.h5
2653/2653 [==============================] - 109s 39ms/step - loss: 0.0057 - mae: 0.0057 - mape: 6.9084 - val_loss: 0.0060 - val_mae: 0.0060 - val_mape: 2.5468
Epoch 2/10
2651/2653 [============================>.] - ETA: 0s - loss: 0.0057 - mae: 0.0057 - mape: 5.8685
Epoch 2: val_loss improved from 0.00600 to 0.00593, saving model to C:\Users\musta\Desktop\ML lab\lab 10\E2-cp-0002-loss0.01.h5
2653/2653 [==============================] - 77s 29ms/step - loss: 0.0057 - mae: 0.0057 - mape: 5.8659 - val_loss: 0.0059 - val_mae: 0.0059 - val_mape: 2.5068
Epoch 3/10
2651/2653 [============================>.] - ETA: 0s - loss: 0.0057 - mae: 0.0057 - mape: 6.7247
Epoch 3: val_loss improved from 0.00593 to 0.00587, saving model to C:\Users\musta\Desktop\ML lab\lab 10\

In [25]:

model = load_model(r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 10\E2-cp-0008-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 11s 20ms/step
Mean Absolute Error (MAE): 97.7
Median Absolute Error (MedAE): 76.93
Mean Squared Error (MSE): 17048.43
Root Mean Squared Error (RMSE): 130.57
Mean Absolute Percentage Error (MAPE): 0.67 %
Median Absolute Percentage Error (MDAPE): 0.53 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
